In [76]:
import pandas as pd
import sqlite3
from num2words import num2words
import os

from datetime import date, timedelta
import datetime
import time

import numpy as np

import re

In [2]:
def sql_reader(filename):
    con = sqlite3.connect(filename)
    df = pd.read_sql_query("SELECT * FROM submissions", con)
    
    
    df = df[['created', 'author', 'title', 'url', 'score', 'num_comments', 'flair_text', 'selftext']]
    return df

In [3]:
xrp = sql_reader('reddit_data\\Ripple\\Ripple.db')

In [8]:
xrp.head()

,created,author,title,url,score,num_comments,flair_text,selftext
0,1255535455,KISteam,Aegon releases $650m of capital,http://www.insuranceerm.com/news-comment/aegon...,1,0,None,
1,1316695135,[deleted],On Ripple,http://newmeraire.blogspot.com/2011/09/on-ripp...,1,0,None,
2,1346189292,60177756,If you're looking for the Ripple payment syste...,http://www.reddit.com/r/ripplers,3,1,None,
3,1361459583,HostFat,Ripple Giveaway!,https://bitcointalk.org/index.php?topic=145506,4,12,None,
4,1363923139,ahbritto,Ripple Meetup,http://www.meetup.com/ripple/,11,0,None,


In [71]:
def merge_data(coinname, df, cachetime):
    
    fName = 'reddit_data\\{}-{}.csv'.format(coinname,cachetime)
    
    if (os.path.isfile(fName)):
        new = pd.read_csv(fName)
    else:          
        reddit = sql_reader('reddit_data\\{}\\{}.db'.format(coinname, coinname))

        new = pd.DataFrame(columns=['Date', 'Reddit'])
        new['Date'] = df['Date']

        oldDate = new['Date'][0]
F
        for i in range(new.shape[0]):
            valsBetween = reddit[reddit['created'] > oldDate]
            valsBetween = valsBetween[reddit['created']  < new['Date'][i]]

            #cols = 'title ' + valsBetween['title'] + ' score ' + valsBetween['score'].apply(lambda x: num2words(x)) + ' comments ' + valsBetween['num_comments'].apply(lambda x: num2words(x))
            cols = 'title ' + valsBetween['title'] + ' score ' + valsBetween['score'].map(str) + ' comments ' + valsBetween['num_comments'].map(str)
            new['Reddit'][i] = (cols.str.cat(sep='\n'))
            oldDate = new['Date'][i]
        
        new.to_csv(fName)
    
    return new

In [ ]:
btc = pd.read_csv('..\\TechnicalAnalysis\\cache\\')#

In [73]:
coins = ['Bitcoin', 'ethereum', 'Monero', 'dashpay', 'dogecoin', 'litecoin', 'Ripple', 'Stellar']

for coin in coins:
    merge_data(coin, df, cachetime='24')

c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [74]:
#now for bitcoin twitter

In [75]:
#tweet, time, likes, replies, retweet. Combined twitter

In [11]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
def extract_content():
    
    start_date = date(2013, 1, 1)
    end_date = date(2017, 12, 31)
    
    combinedDf = pd.DataFrame(columns=['Tweet', 'Time', 'Likes', 'Replies', 'Retweet'])
    
    for single_date in daterange(start_date, end_date):
        d = str(single_date.strftime("%Y-%m-%d"))
        
        try:
            df = pd.read_csv('twitter_data\\bitcoin\\{}.csv'.format(d)).drop('Unnamed: 0', axis=1)
            combinedDf = pd.concat([combinedDf, df[['Tweet', 'Time', 'Likes', 'Replies', 'Retweet']]]).reset_index(drop=True)

        except:
            pass
    
    combinedDf.to_csv('twitter_data\\bitcoin\\combined.csv')

In [9]:
df = pd.read_csv('twitter_data\\bitcoin\\combined.csv')

In [45]:
df['Time'] = df['Time'].replace(to_replace=0, method='ffill')

In [49]:
def return_time(x):
    try:
        y = time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timetuple())
                        
    except:
        y = 0
                        
    return int(y)

In [51]:
df['Time'] = df.fillna(method='ffill')['Time'].apply(return_time)

In [53]:
df['Time'] = df['Time'].replace(to_replace=0, method='ffill')

In [54]:
df = df.fillna(method='ffill')

In [59]:
df['Time'] = df['Time'].astype(int)

In [61]:
df.head()

,Unnamed: 0,Tweet,Time,Likes,Replies,Retweet
0,0,17kc7ncv just won 8.551 Bitcoins at http://Sat...,1357064068,0.0,0.0,0.0
1,1,Is there any way to track the holder of a Bitc...,1357063953,0.0,0.0,0.0
2,2,Bitcoin in danger of becoming a fiat currency?...,1357063953,0.0,0.0,0.0
3,3,Sign Up For a Free Bitcoin Wallet @ http://Blo...,1357063941,0.0,0.0,0.0
4,4,3`081 S.DICE @ 0.00370000BTC = 11.39970000BTC ...,1357063834,0.0,0.0,0.0


In [60]:
df.to_csv('twitter_data\\bitcoin\\time.csv')

In [7]:
from nltk.corpus import stopwords
from gensim.models.doc2vec import TaggedDocument 
from gensim import utils
from gensim.models import Doc2Vec

c:\programdata\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
def cleanData(data):
    pattern = [ 'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs'
                '([^a-zA-Z0-9 ]+?)', #anything else except text
                ]

    sub_pattern = re.compile('|'.join(pattern))
    common_w = set(stopwords.words("english"))
    
    if isinstance(data, pd.Series):
        
        data = data.str.lower()
        
        for idx, row in enumerate(data):
            splitRow = row.split(' ')
            splitRow = [w for w in splitRow if not w in common_w] 

            data.iloc[idx] = " ".join(splitRow)
            
        replaced = data.str.replace(sub_pattern, '').str.strip()
    else:

        data = data.lower()
        splitted = data.split(' ')

        newSplit = [w for w in splitted if not w in common_w] 
        word = " ".join(newSplit)
        replaced = re.sub(sub_pattern, '', word).strip()
        
    return replaced

In [78]:
def merge_twitter_data(df):
    
    twitter = pd.read_csv('twitter_data\\bitcoin\\time.csv')
    
    new = pd.DataFrame(columns=['Date', 'Twitter'])
    new['Date'] = df['Date']

    oldDate = new['Date'][0]
    
    for i in range(new.shape[0]):
        valsBetween = twitter[twitter['Time'] > oldDate]
        valsBetween = valsBetween[twitter['Time']  < new['Date'][i]]

        #cols = 'title ' + valsBetween['title'] + ' score ' + valsBetween['score'].apply(lambda x: num2words(x)) + ' comments ' + valsBetween['num_comments'].apply(lambda x: num2words(x))
        cols = 'tweet ' + cleanData(valsBetween['Tweet']) + ' likes ' + valsBetween['Likes'].map(str) + ' replies ' + valsBetween['Replies'].map(str) + ' retweets ' + valsBetween['Retweet'].map(str)
        new['Twitter'][i] = (cols.str.cat(sep='\n'))
        oldDate = new['Date'][i]

    new.to_csv('twitter_data\\bitcoin\\readable.csv')


    return new

In [69]:
t = pd.read_csv('reddit_data\\Bitcoin-24.csv')

In [79]:
merge_twitter_data(t)

c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Date,Twitter
0,1439010000,
1,1439096400,tweet average price bitcoin across exchanges 2...
2,1439182800,tweet antminer s3 bitcoin miner 441 ghs bitco...
3,1439269200,tweet latest bitcoin price index 26375 usd pic...
4,1439355600,tweet rising fintech investment asia suggest r...
5,1439442000,tweet bitcoin amazon web service outage shows ...
6,1439528400,tweet norway rocks invest turn crowdfund bitco...
7,1439614800,tweet foot bitcoin pixar anger via fomocoin l...
8,1439701200,tweet artbyte store sell music artbytes bitc...
9,1439787600,tweet fiatliberty thanks coming board endorsem...


In [80]:
def gensim_sentence(data):
    labeledRow=[]

    for idx, row in data.iteritems():
        labeledRow.append(TaggedDocument(utils.to_unicode(row).split(), ['Text' + '_%s' % str(idx)]))

    return labeledRow

In [83]:
twitterData = pd.read_csv('twitter_data\\bitcoin\\readable.csv')

In [85]:
twitterData.drop('Unnamed: 0', axis=1)

,Date,Twitter
0,1439010000,NaN
1,1439096400,tweet average price bitcoin across exchanges 2...
2,1439182800,tweet antminer s3 bitcoin miner 441 ghs bitco...
3,1439269200,tweet latest bitcoin price index 26375 usd pic...
4,1439355600,tweet rising fintech investment asia suggest r...
5,1439442000,tweet bitcoin amazon web service outage shows ...
6,1439528400,tweet norway rocks invest turn crowdfund bitco...
7,1439614800,tweet foot bitcoin pixar anger via fomocoin l...
8,1439701200,tweet artbyte store sell music artbytes bitc...
9,1439787600,tweet fiatliberty thanks coming board endorsem...


In [86]:
def get_doc2vec(inp):
    labeledSentences = gensim_sentence(inp)
    
    fname='doc2vectwitter.d2v'
    
    if os.path.isfile(fname):
        docModel = Doc2Vec.load(fname)
    else:
        docModel = Doc2Vec(min_count=0, window=5, vector_size=300, sample=1e-4, negative=5, workers=4, epochs=5, seed=1)
        docModel.build_vocab(labeledSentences)
        docModel.train(labeledSentences, total_examples=docModel.corpus_count, epochs=docModel.iter)
        docModel.save(fname)
    
    return docModel

In [89]:
twitterData['Twitter'] = twitterData['Twitter'].fillna(value='')

In [93]:
twiterdoc = get_doc2vec(twitterData['Twitter'])

In [96]:
def process_data(docModel, out):
    X = np.zeros((out.shape[0], 300))
    
    for i in range(out.shape[0]):
        X[i] = docModel.docvecs['Text_'+str(i)]
    
    return X

In [100]:
twitterFeatures = pd.DataFrame(process_data(twiterdoc, twitterData['Twitter']))

In [101]:
twitterFeatures['Date'] = twitterData['Date']

In [104]:
twitterFeatures = twitterFeatures.set_index('Date')

In [106]:
twitterFeatures.to_csv('twitter_data\\bitcoin\\twitterFeatures.csv')

In [107]:
def get_doc2vec_reddit(inp, coinname):
    labeledSentences = gensim_sentence(inp)
    
    fname='doc2vecreddit-{}.d2v'.format(coinname)
    
    if os.path.isfile(fname):
        docModel = Doc2Vec.load(fname)
    else:
        docModel = Doc2Vec(min_count=0, window=5, vector_size=300, sample=1e-4, negative=5, workers=4, epochs=5, seed=1)
        docModel.build_vocab(labeledSentences)
        docModel.train(labeledSentences, total_examples=docModel.corpus_count, epochs=docModel.iter)
        docModel.save(fname)
    
    return docModel

In [109]:
coins = ['Bitcoin', 'ethereum', 'Monero', 'dashpay', 'dogecoin', 'litecoin', 'Ripple', 'Stellar']

bigDf = pd.DataFrame(columns=['Date', 'Reddit'])

doc2vecs = {}

for coin in coins:
    df = pd.read_csv('reddit_data//{}-24.csv'.format(coin))
    doc2vecs[coin] = get_doc2vec_reddit(df['Reddit'].fillna(value=''), coin) 

c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  # This is added back by InteractiveShellApp.init_path()


In [110]:
redditFeatures = {}

for coin in coins:
    redditFeatures[coin] = pd.DataFrame(process_data(doc2vecs[coin], pd.read_csv('reddit_data//{}-24.csv'.format(coin))['Reddit']))
    redditFeatures[coin]['Date'] = twitterData['Date']
    redditFeatures[coin] = redditFeatures[coin].set_index('Date')

    redditFeatures[coin].to_csv('reddit_data\\\\{}Features.csv'.format(coin))